In [ ]:
# Ingest from KEGG API
# KEGG API documentation: https://www.kegg.jp/kegg/rest/keggapi.html
import requests

from typing import Optional

BASE_URL = "https://rest.kegg.jp"


def fetch_kegg_data(
    endpoint: str,
    entries: str,
    *,
    timeout: int = 20,
    retries: int = 3,
    session: Optional[requests.Session] = None,
) -> str:
    """
    Generic KEGG REST API fetcher.

    Supports all KEGG endpoints:
        get, list, link, find, conv, etc.

    Example calls:
        fetch_kegg_data("get", "hsa00010")
        fetch_kegg_data("link", "reaction/hsa00010")
        fetch_kegg_data("get", "R00710")
        fetch_kegg_data("list", "pathway/hsa")

    Args:
        endpoint: KEGG REST endpoint.
        entries: Entry identifier or query string.
        timeout: HTTP timeout seconds.
        retries: Retry attempts.
        session: Optional shared requests session.

    Returns:
        Raw response text or empty string on failure.
    """

    url = f"{BASE_URL}/{endpoint}/{entries}"
    sess = session or requests.Session()

    for attempt in range(1, retries + 1):
        try:
            response = sess.get(url, timeout=timeout)
            response.raise_for_status()
            return response.text

        except requests.exceptions.RequestException as e:
            if attempt == retries:
                print(f"[KEGG ERROR] {url} -> {e}")
                return ""
            else:
                print(f"[KEGG RETRY {attempt}/{retries}] {url}")




In [11]:
response = fetch_kegg_data("link", "reaction/hsa00010")
response

'\n'

In [5]:
import re

def extract_reactions(text):
    """Extract unique reaction IDs from KEGG pathway text."""
    reactions = re.findall(r'R\d+', text)
    return list(set(reactions))

In [6]:
extract_reactions(response)

['R1']